In [ ]:
!pip install alpaca_trade_api

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.7/323.7 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 682.2/682.2 kB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.1/175.1 kB 18.9 MB/s eta 0:00:00
Reason for being yanked: This version includes overly restrictive multidict upper boundary disallowing multidict v6+. The previous patch version didn't have that and this is now causing dependency resolution problems for the users who have an "incompatible" version pinned. This is not really necessary anymore and will be addressed in the next release v3.8.3

https://github.com/aio-libs/aiohttp/pull/6950
  Attempting uninstall: msgpack
    Found existing installation: msgpack 1.0.7
    Uninstalling msgpac

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
import alpaca_trade_api as tradeapi

# API keys
alpaca_api_key = "PKJYB527YAQVS4J907RJ"
alpaca_secret_key = "zb5bvTpu8cWaKIgAbDCESouV9MX0qn4RXiQzS5n8"

# Create the Alpaca API object
alpaca = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version="v2")

# Set the tickers and timeframe
tickers = ["NVDA"]
timeframe = "1Day"

# Set the date range
start = pd.Timestamp("2014-02-07", tz="America/New_York").isoformat()
end = pd.Timestamp("2020-02-07", tz="America/New_York").isoformat()

# Get the data
df = alpaca.get_bars(tickers, timeframe, start=start, end=end).df

# Drop the 'symbol' column and add shifted close column
df = df.drop(columns=['symbol'])

# Function to create the signal
def signal_creator_5(row):
    percent_diff = (row['close'] - row['shifted_close_5']) / row['shifted_close_5']
    if percent_diff > 0.03:
        return 1
    elif percent_diff < -0.03:
        return -1
    else:
        return 0

def signal_creator_15(row):
    percent_diff = (row['close'] - row['shifted_close_15']) / row['shifted_close_15']
    if percent_diff > 0.03:
        return 1
    elif percent_diff < -0.03:
        return -1
    else:
        return 0

def signal_creator_30(row):
    percent_diff = (row['close'] - row['shifted_close_30']) / row['shifted_close_30']
    if percent_diff > 0.03:
        return 1
    elif percent_diff < -0.03:
        return -1
    else:
        return 0

# 5 day shift
df['shifted_close_5'] = df['close'].shift(periods=5)
df['signal_5'] = 0
df['signal_5'] = df.apply(signal_creator_5, axis=1)

# 15 day shift
df['shifted_close_15'] = df['close'].shift(periods=15)
df['signal_15'] = 0
df['signal_15'] = df.apply(signal_creator_15, axis=1)

# 30 day shift
df['shifted_close_30'] = df['close'].shift(periods=30)
df['signal_30'] = 0
df['signal_30'] = df.apply(signal_creator_30, axis=1)

# Columns and NA drops to prevent data leakege
df.dropna(inplace=True)
df.drop(columns=['shifted_close_5','shifted_close_15','shifted_close_30'],inplace=True)
display(df.head(50))

# Separate features and target
features = df.drop(columns = ['signal_5', 'signal_15', 'signal_30'], axis=1)
target = df.drop(columns = ['close', 'high', 'low', 'trade_count', 'open', 'volume', 'vwap'], axis=1)
display(target)


In [ ]:
# # Scale the features
# scaler = MinMaxScaler(feature_range=(0, 1))
# scaled_features = scaler.fit_transform(features)

# # Function to create dataset for LSTM
# def create_dataset(X, y, time_steps=5):
#     Xs, ys = [], []
#     for i in range(len(X) - time_steps):
#         Xs.append(X[i:(i + time_steps), :])
#         ys.append(y[i + time_steps])
#     return np.array(Xs), np.array(ys)

# # Creating the dataset
# time_steps = 5
# X, y = create_dataset(scaled_features, target.values, time_steps)

# # Splitting data into training and testing sets
# train_size = int(len(X) * 0.8)
# X_train, X_test = X[:train_size], X[train_size:]
# y_train, y_test = y[:train_size], y[train_size:]

# # Building the LSTM model
# model_5 = Sequential()
# model_15 = Sequential()
# model_30 = Sequential()
# def builder(model):
#   model.add(LSTM(50, return_sequences=True, input_shape=(time_steps, X.shape[2])))
#   model.add(LSTM(50, return_sequences=False))
#   model.add(Dense(25))
#   model.add(Dense(1))

# builder(model_5)
# builder(model_15)
# builder(model_30)

# # Compiling the model
# model_5.compile(optimizer='adam', loss='mean_squared_error')
# model_15.compile(optimizer='adam', loss='mean_squared_error')
# model_30.compile(optimizer='adam', loss='mean_squared_error')

# # Training the model
# model_5.fit(X_train, y_train, epochs=100, batch_size=32)
# model_15.fit(X_train, y_train, epochs=100, batch_size=32)
# model_30.fit(X_train, y_train, epochs=100, batch_size=32)

# # Model Evaluation
# def eval(model):
#   test_loss = model.evaluate(X_test, y_test)
#   print(f'Test Loss: {test_loss}')

# eval(model_5)
# eval(model_15)
# eval(model_30)

# # Making Predictions
# predictions_5 = model_5.predict(X_test)
# predictions_15 = model_15.predict(X_test)
# predictions_30 = model_30.predict(X_test)

# # Function to interpret predictions
# def interpret_prediction(prediction):
#     if prediction > 0.03:
#         return 1  # Buy
#     elif prediction < -0.03:
#         return -1  # Sell
#     else:
#         return 0  # Hold

# interpret_prediction(predictions_5)
# interpret_prediction(predictions_15)
# interpret_prediction(predictions_30)

# #interpreted_signals = [interpret_prediction(p[0]) for p in predictions]

# # Example: Print first 10 decisions
# print(interpreted_signals[:10])

In [67]:
# Scale the features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_features = scaler.fit_transform(features)

In [68]:
# Function to create dataset for LSTM
def create_dataset(X, y, time_steps=5):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:(i + time_steps), :])
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)

In [69]:
# Creating the dataset
time_steps = 5
X, y = create_dataset(scaled_features, target.values, time_steps)

In [70]:
# Splitting data into training and testing sets
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [71]:
model_5 = Sequential()

In [72]:
model_15 = Sequential()

In [73]:
model_30 = Sequential()

In [74]:
# Building the LSTM model
def builder(model):
  model.add(LSTM(50, return_sequences=True, input_shape=(time_steps, X.shape[2])))
  model.add(LSTM(50, return_sequences=False))
  model.add(Dense(25))
  model.add(Dense(1))

In [75]:
builder(model_5)

In [76]:
builder(model_15)

In [77]:
builder(model_30)

In [78]:
# Compiling the model
model_5.compile(optimizer='adam', loss='mean_squared_error')
model_15.compile(optimizer='adam', loss='mean_squared_error')
model_30.compile(optimizer='adam', loss='mean_squared_error')


In [79]:
# Training the model
model_5.fit(X_train, y_train, epochs=50, batch_size=32)

Epoch 1/50
25/25 [==============================] - 6s 10ms/step - loss: 0.6651
Epoch 2/50
25/25 [==============================] - 0s 10ms/step - loss: 0.6153
Epoch 3/50
25/25 [==============================] - 0s 11ms/step - loss: 0.5952
Epoch 4/50
25/25 [==============================] - 0s 12ms/step - loss: 0.5922
Epoch 5/50
25/25 [==============================] - 0s 11ms/step - loss: 0.5876
Epoch 6/50
25/25 [==============================] - 0s 10ms/step - loss: 0.5869
Epoch 7/50
25/25 [==============================] - 0s 11ms/step - loss: 0.5816
Epoch 8/50
25/25 [==============================] - 0s 11ms/step - loss: 0.5778
Epoch 9/50
25/25 [==============================] - 0s 11ms/step - loss: 0.5775
Epoch 10/50
25/25 [==============================] - 0s 11ms/step - loss: 0.5679
Epoch 11/50
25/25 [==============================] - 0s 10ms/step - loss: 0.5636
Epoch 12/50
25/25 [==============================] - 0s 10ms/step - loss: 0.5598
Epoch 13/50
25/25 [==================

In [80]:
model_15.fit(X_train, y_train, epochs=50, batch_size=32)

Epoch 1/50
25/25 [==============================] - 6s 17ms/step - loss: 0.6668
Epoch 2/50
25/25 [==============================] - 0s 16ms/step - loss: 0.6201
Epoch 3/50
25/25 [==============================] - 0s 15ms/step - loss: 0.5999
Epoch 4/50
25/25 [==============================] - 0s 16ms/step - loss: 0.5890
Epoch 5/50
25/25 [==============================] - 0s 16ms/step - loss: 0.5846
Epoch 6/50
25/25 [==============================] - 0s 13ms/step - loss: 0.5822
Epoch 7/50
25/25 [==============================] - 0s 11ms/step - loss: 0.5745
Epoch 8/50
25/25 [==============================] - 0s 11ms/step - loss: 0.5712
Epoch 9/50
25/25 [==============================] - 0s 12ms/step - loss: 0.5628
Epoch 10/50
25/25 [==============================] - 0s 12ms/step - loss: 0.5599
Epoch 11/50
25/25 [==============================] - 0s 11ms/step - loss: 0.5906
Epoch 12/50
25/25 [==============================] - 0s 11ms/step - loss: 0.5608
Epoch 13/50
25/25 [==================

In [81]:
model_30.fit(X_train, y_train, epochs=50, batch_size=32)

Epoch 1/50
25/25 [==============================] - 5s 10ms/step - loss: 0.6549
Epoch 2/50
25/25 [==============================] - 0s 11ms/step - loss: 0.6048
Epoch 3/50
25/25 [==============================] - 0s 12ms/step - loss: 0.5942
Epoch 4/50
25/25 [==============================] - 0s 12ms/step - loss: 0.5906
Epoch 5/50
25/25 [==============================] - 0s 11ms/step - loss: 0.5893
Epoch 6/50
25/25 [==============================] - 0s 10ms/step - loss: 0.5815
Epoch 7/50
25/25 [==============================] - 0s 11ms/step - loss: 0.5813
Epoch 8/50
25/25 [==============================] - 0s 10ms/step - loss: 0.5753
Epoch 9/50
25/25 [==============================] - 0s 10ms/step - loss: 0.5701
Epoch 10/50
25/25 [==============================] - 0s 11ms/step - loss: 0.5725
Epoch 11/50
25/25 [==============================] - 0s 10ms/step - loss: 0.5782
Epoch 12/50
25/25 [==============================] - 0s 10ms/step - loss: 0.5653
Epoch 13/50
25/25 [==================

In [82]:
# Model Evaluation
def eval(model):
  test_loss = model.evaluate(X_test, y_test)
  print(f'Test Loss: {test_loss}')


In [83]:
eval(model_5)

7/7 [==============================] - 1s 6ms/step - loss: 0.6192
Test Loss: 0.6192113757133484


In [84]:
eval(model_15)

7/7 [==============================] - 2s 5ms/step - loss: 0.6656
Test Loss: 0.6655747890472412


In [85]:
eval(model_30)

7/7 [==============================] - 1s 4ms/step - loss: 0.5088
Test Loss: 0.5088016390800476


In [86]:
# Making Predictions
predictions_5 = model_5.predict(X_test)

7/7 [==============================] - 1s 5ms/step


In [87]:
predictions_15 = model_15.predict(X_test)

7/7 [==============================] - 1s 4ms/step


In [88]:
predictions_30 = model_30.predict(X_test)

7/7 [==============================] - 1s 4ms/step


In [89]:
# Function to interpret predictions
def interpret_prediction(prediction):
    if prediction > 0.03:
        return 1  # Buy
    elif prediction < -0.03:
        return -1  # Sell
    else:
        return 0  # Hold

#interpreted_signals = [interpret_prediction(p[0]) for p in predictions]

# Example: Print first 10 decisions




In [90]:
results_5 = [interpret_prediction(p[0]) for p in predictions_5]
print(results_5[:-25])

[1, 1, -1, -1, -1, -1, 0, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, -1, -1, -1, -1, -1, 1, 1, 1, 0, -1, 0, 1, 1, 1, 1, 1, 0, -1, -1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, -1, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1, 1, 1, 1, 1, -1, 0, 1, 1, 1, 1, 1, 1, 0, -1, 1, 1, 1, 1, 1, 0, 1, 1, -1, -1, -1, -1, -1, -1, 0, 1, 1, 1, 1, -1, -1, -1, -1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [91]:
results_15 = [interpret_prediction(p[0]) for p in predictions_15]
print(results_15[:-25])

[1, 1, -1, -1, -1, -1, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, -1, -1, -1, -1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, -1, 0, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 1, 1, 1, 1, 0, -1, 0, 1, 1, 1, 1, 1, 1, -1, 0, 1, 1, 1, 1, 0, 0, 1, 0, -1, -1, -1, -1, -1, -1, -1, 0, 1, 1, 1, -1, -1, -1, -1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0]


In [92]:
results_30 = [interpret_prediction(p[0]) for p in predictions_30]
print(results_30[:-25])

[1, 1, 1, 0, 1, 1, 1, 1, 1, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 1, 0, 0, 0, -1, -1, -1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, -1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
